In [14]:
# Setup for Colab: clone repo and install dependencies
import sys
if 'google.colab' in sys.modules:
    !rm -rf your-repo   # remove old copy
    !git clone https://github.com/cwallmeier/Technasium/tree/main/AnalyticalModel.git
    !pip install -r requirements.txt

In [15]:
# Import standard python packages to handle maths and graphics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from CoolProp.CoolProp import PropsSI

print('Imports completed')

Imports completed


In [16]:
{
    "tags": [
        "hide-cell",
    ]
}

"""
Created on Mon Feb 13 15:28:23 2023

@author: adaniilidis, cwallmeier
"""

import pandas as pd
import numpy as np
from CoolProp.CoolProp import PropsSI


class make_doublet():

    def __init__(self,
                 # Input parameters Geometry
                 depth_m=2283.33334,  # top reservoir depth in meters
                 thickn_m=100,  # reservoir thickness in meters

                 # Input parameters rock properties
                 poro=25,  # reservoir posority in percent
                 perm_mD=300,  # reservoir permeability mD
                 rho_rock=2300,  # rock density kg/m3
                 Cp_rock=1,  # specific heat capacity rock kJ/(kgK)

                 # Input parameters fluid properties
                 rho_fluid=1000,  # fluid density kg/m3
                 Cp_fluid=4.180,  # specific heat capacity fluid kJ/(kgK)

                 # Input parameters pT
                 T_surf=10,  # refrence temperature at surface (degC)
                 T_grad=30,  # temperature gradient (degC/km)
                 p_surf=101325,  # reference pressure at surface (Pa)
                 p_grad=9792100,  # pressure gradient (Pa/km)

                 # Input parameters wells
                 q_m3_h=200,  # volume flow rate m3/h
                 w_space=1200,   # well spacing in meters
                 w_diam=0.2032,  # well diameter in meters
                 T_inj=30,  # injection temperature degC

                 # Input parameters economic
                 pump_eta=0.5,  #
                 ):
        # Input parameters Geometry
        self.r_d = depth_m
        self.r_h = thickn_m

        # Input parameters rock properties
        self.poro = poro / 100
        self.rho_rock = rho_rock
        self.Cp_rock = Cp_rock
        self.perm_mD = perm_mD
        self.perm_m2 = self.perm_mD * 9.8692326671601e-13 * 1e-3  # reservoir permeability in m2

        # Input parameters fluid properties
        self.rho_fluid = rho_fluid
        self.Cp_fluid = Cp_fluid

        # Input parameters pT
        self.T_ref = T_surf
        self.T_grad = T_grad
        self.p_ref = p_surf
        self.p_grad = p_grad

        # Input parameters wells
        self.q_m3_h = q_m3_h
        self.q_m3_s = q_m3_h / 3600  # seconds
        self.w_space = w_space
        self.w_diam = w_diam
        self.T_inj = T_inj
        self.p_prod = self.p_ref + ((self.r_d + self.r_h / 2) * 1e-3 * self.p_grad)  # undisturbed p at reservoir depth
        self.T_prod = self.T_ref + ((self.r_d + self.r_h / 2) * 1e-3 * self.T_grad)  # production temperature degC
        self.mu_0 = PropsSI('viscosity', 'T', self.T_prod + 273.15, 'P', self.p_prod, 'Water')
        self.mu_inj = PropsSI('viscosity', 'T', self.T_inj + 273.15, 'P', self.p_prod, 'Water')

        # Input parameters economic
        self.pump_eta = pump_eta

    def lmbda(self):
        mobility_lambda = self.poro * self.rho_fluid * self.Cp_fluid / (
                (1 - self.poro) * self.rho_rock * self.Cp_rock + self.rho_fluid * self.Cp_fluid * self.poro)
        return mobility_lambda

    def mu(self, r):
        """
        :param r: position along doublet-line in m, measured from injector towards producer
        :return: viscosity in Pas at that position
        """
        if r < 0:
            mu = self.mu_inj
        elif r > self.w_space:
            mu = self.mu_0
        else:
            ratio = r / self.w_space
            mu = self.mu_inj * (1 - ratio) + self.mu_0 * ratio
        return mu

    def dp_wells(self):

        c_inj = self.mu_inj / self.perm_m2 * self.q_m3_s / (2 * np.pi * self.r_h)
        c_prd = self.mu_0 / self.perm_m2 * self.q_m3_s / (2 * np.pi * self.r_h)

        self.dp_MPa = np.log((self.w_space - self.w_diam / 2) / self.w_diam / 2) * (c_inj + c_prd) * 1e-6

        return self.dp_MPa

    def t_breakthrough(self):
        """
        :return: arrival time of cold front at producer well
        """
        self.t_cold_yrs = (self.poro / self.lmbda() *
                           (2 * np.pi * self.r_h) / self.q_m3_h * self.w_space ** 2 / 6) / (365 * 24)
        return self.t_cold_yrs

    def p_pumps(self):
        self.p_pumps_MW = self.dp_wells() * self.q_m3_s / self.pump_eta
        return self.p_pumps_MW

    def p_doublet(self):
        """ power produced by doublet in MW """
        self.P_doublet_kW = self.q_m3_s * self.rho_fluid * self.Cp_fluid * (self.T_prod - self.T_inj)
        self.P_doublet_MW = self.P_doublet_kW * 1e-3

        return self.P_doublet_MW


# "Creating" the doublet
In the next cell, we decribe the subsurface and the geothermal doublet. Then we "create" the digital doublet. It is called *my_doublet*.

(make_doublet() has some more settings that you can change, if you want to. Look into the file doublet.py to see the whole list.)

In [17]:
# Reservoir depth: The depth of the top of the reservoir in m.
res_depth = 2300

# Reservoir thickness in m.
res_thick = 80

# porosity in percent
porosity=20

# Permeability in mDarcy
res_perm = 600

# well spacing. The distance between injector and producer in m
wells_x = 800

# injection temperature. the temperature (in deg Celsius) of the cold water that is being re-injected.
injT = 32

# Give all this input to the function:
doublet1 = make_doublet(depth_m=res_depth, thickn_m=res_thick, poro=porosity, perm_mD=res_perm, w_space=wells_x, T_inj=injT)


print('Doublet created')

Doublet created


# Physics
Now we will look at the physics of the doublet. We will look at 3 things

1. Production temperature.
2. Lifetime of the doublet*.
3. The power (in Watt or kWh) that is needed to run the system.**

\* "Lifetime" describes how long the water that comes out of the production well stays warm. This is usually many years or decades. After that, it cools down slowly because the re-injected water has travelled all the way from the injector to the producer.

\*\* Most of the time, we talk about how much power (=energy per time) a geothermal doublet produces. But we also have to put in some power to run the pumps. The difference, "power_out - power_in" is how much we actually got.

In [18]:
# Production temperature
Temp_prod = doublet1.T_prod
print('T_out: %s °C' % round(Temp_prod, 2))

T_out: 80.2 °C


In [19]:
# time of thermal breakthrough
time_breakth = doublet1.t_breakthrough()
print('Time of thermal breakthrough: %s years' % round(time_breakth, 2))

Time of thermal breakthrough: 19.59 years


In [20]:
# Power

# 1) How much power do we generate?
p_out = doublet1.p_doublet()
print('Power produced by the doublet: %s MW'  %round(p_out, 2))

# 2) how much power is needed to run the pumps?
p_in = doublet1.p_pumps()
print('Power consumed by the doublet: %s MW' %round(p_in, 2))

# 3) The difference:
p_net = p_out - p_in
print('Net power produced: %s MW' %round(p_net, 2))

Power produced by the doublet: 11.19 MW
Power consumed by the doublet: 0.18 MW
Net power produced: 11.02 MW


### The other files in this directory
- *doublet.py* contains the equations used in this notebook.
  - you can do this entire exercise without looking into *doublet.py*
  - but feel free to open it and explore

### Problems?
- if you changed some numbers, but the result does not change: --> Look for the option to *Restart Kernel*


